# NumPy-like Operations

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)
@[Chaoming Wang](https://github.com/chaoming0625)

## `brainpy.math.array`

Neural modeling often involves the computation of thousands or even millions of elements and requires large-scale computation. To support this, BrainPy provides a basic data structure `brainpy.math.array` with abundant operations. In fact, the concept of `brainpy.math.array` is the same as `ndarray` in NumPy, and the operations used in BrainPy closely resemble NumPy operations. 

```{note}
Users who are not familiar with the Python package NumPy may refer to [https://numpy.org/](https://numpy.org/) for more information.
```

## NumPy-like operations for arrays

For `brainpy.math.array`, BrainPy provides array-related operations that have the same form as NumPy while capable of performing JIT transformations with the help of JAX. Almost all NumPy operations can be used in BrainPy as long as users transform `numpy.xxx` to `brainpy.math.xxx`.

Specifically, the NumPy-like operations for `brainpy.math.array` include:

1. Creating an array: users can use `bm.array()`， `bm.ones()`， `bm.zeros()`， `bm.arange()`， and so on to create an array.
2. Element-wise operations: `+`, `-`, `*`, `/`, ...
3. Aggregation functions: `.max()`, `.min()`, `.sum()`, `.mean()`, `.prod()`, ...
4. Broadcasting: operating two arrays that have different dimensions (usually a smaller array is broadcast to a larger array with compatible shapes to complete operation).
5. Indexing, slicing, and iterating: they are the same operations done on sequence data structures such as lists.
6. Other mathematical and logical functions: `bm.sin()`, `bm.cos()`, `bm.sort()`, `bm.argmax()`, `bm.where()`, ...

In [11]:
import brainpy.math as bm

bm.set_platform('cpu')

Here are some examples of array-related operations in BrainPy:

In [12]:
# to create an array
a1 = bm.array([[0, 1, 2], [3, 4, 5]])
a1

Array(value=DeviceArray([[0, 1, 2],
                         [3, 4, 5]]),
      dtype=int32)

In [13]:
# element-wise operation
a1 += 1
a1

Array(value=DeviceArray([[1, 2, 3],
                         [4, 5, 6]]),
      dtype=int32)

In [14]:
a2 = bm.ones(3)
a2

Array(value=DeviceArray([1., 1., 1.]), dtype=float32)

In [15]:
# slicing
a2[0:2] = 10
a2

Array(value=DeviceArray([10., 10.,  1.]), dtype=float32)

In [16]:
# broadcasting
a3 = a1 + a2
a3

Array(value=DeviceArray([[11., 12.,  4.],
                         [14., 15.,  7.]]),
      dtype=float32)

In [17]:
# sorting
bm.sort(a3)

Array(value=DeviceArray([[ 4., 11., 12.],
                         [ 7., 14., 15.]]),
      dtype=float32)

In [18]:
# reshaping
a4 = a1.reshape(6, -1)
a4

Array(value=DeviceArray([[1],
                         [2],
                         [3],
                         [4],
                         [5],
                         [6]]),
      dtype=int32)

Besides all the operations mentioned above, BrainPy also offers NumPy-like random functions that are different from JAX. To enable random functions in JIT compilation, JAX requires users to pass an explicit key to define a random state, which is different from NumPy. Here is an example for random number generation in JAX:

In [23]:
from jax import random

key = random.PRNGKey(0)  # define a key explicitly
r1 = random.uniform(key)  # a random number generated from a uniform distribution
r2 = random.uniform(key)  # reusing the same key leads to the same result
r1, r2

(DeviceArray(0.41845703, dtype=float32),
 DeviceArray(0.41845703, dtype=float32))

In BrainPy, however, users can use random functions in the same way as in NumPy, but with an extraordinary speed comparable to JAX:

In [29]:
r3 = bm.random.uniform()
r4 = bm.random.uniform(key=1)  # users can also assign an explicit key to generate a constant result
r5 = bm.random.uniform(key=1)
r3, r4, r5

(DeviceArray(0.9309926, dtype=float32),
 DeviceArray(0.11815023, dtype=float32),
 DeviceArray(0.11815023, dtype=float32))

Users can also generate a random array using random functions in BrainPy:

In [34]:
bm.random.random((5, 3))  # to generate random floats number in the interval [0.0, 1.0)

Array(value=DeviceArray([[0.33878195, 0.6672709 , 0.37249923],
                         [0.4169147 , 0.7702857 , 0.28928375],
                         [0.23907256, 0.08634043, 0.11700511],
                         [0.37935913, 0.6523317 , 0.15176404],
                         [0.6398171 , 0.7524489 , 0.18694246]]),
      dtype=float32)

For all array-related operations, please refer to the [`brianpy.math` module](../apis/auto/math.html).

## When to use `brainpy.math.array`?

Simply speaking, users can use `brainpy.math.array` whenever array computation is involved in JIT compilation. While JIT compilation speeds up code execution, it also brings some limitations. Once an array is given to the JIT compiler, the values inside the array cannot be changed. This means `brainpy.math.array` can only be used to store values that do not change over time. Static neural properties such as the membrane capacitance, the voltage threshold, and the time constant of a neuron population, and static synaptic connections between neurons are good candidates to be defined as arrays (or a unified single value).

To extend arrays to store changing values, BrainPy offers a new data structure named ``brainpy.math.Variable``. More details about Variables can be found in [`Variable` and `BrainPbject`](variables.ipynb).